In [1]:
import feature_selection_module2 as fsm
import os
import pandas as pd

from expttools import Experiment

pip install 'aif360[LawSchoolGPA]'
2023-02-06 05:29:57.421957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 05:29:57.546662: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-06 05:29:57.551865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /modules/apps/julia/1.7.2/lib
2023-02-06 05:29:57.551877: I t

In [2]:

from expttools import ExperimentResult

In [3]:
class NamedDataset():
    def __init__(self,name,df):
        self.name = name
        self.df = df
        
    def copy(self):
        return self

In [4]:
def name_function(cur_param):
    #print(cur_param)
    name_parts = []
    name_parts.append(cur_param['data'].name) 
    # some short version of features
    name_parts.append(cur_param['technique'].__name__)
    name_parts.append(cur_param['model'].__name__)
    name_parts.append(cur_param['p_att_description'][0])
    name_parts.append(str(cur_param['greater']))
    name_parts.append(str(cur_param['k']))
    #print(name_parts)
    return '_'.join(name_parts)

In [5]:

adult_df = pd.read_csv('adult_dataset.csv')
adult_df.drop('Unnamed: 0', axis = 1, inplace = True)
adult = adult_df.copy()
adult['age_bins'] = pd.cut(adult['age'],10)
adult['age_bins'] = adult['age_bins'].astype(str)
adult.head()


,age,education-num,sex,capital-gain,capital-loss,hours-per-week,income-per-year,education,marital-status,native-country,occupation,race,relationship,workclass,age_bins
0,25.0,7.0,1.0,0.0,0.0,40.0,0.0,11th,Never-married,United-States,Machine-op-inspct,Black,Own-child,Private,"(24.3, 31.6]"
1,38.0,9.0,1.0,0.0,0.0,50.0,0.0,HS-grad,Married-civ-spouse,United-States,Farming-fishing,White,Husband,Private,"(31.6, 38.9]"
2,28.0,12.0,1.0,0.0,0.0,40.0,1.0,Assoc-acdm,Married-civ-spouse,United-States,Protective-serv,White,Husband,Local-gov,"(24.3, 31.6]"
3,44.0,10.0,1.0,7688.0,0.0,40.0,1.0,Some-college,Married-civ-spouse,United-States,Machine-op-inspct,Black,Husband,Private,"(38.9, 46.2]"
4,34.0,6.0,1.0,0.0,0.0,30.0,0.0,10th,Never-married,United-States,Other-service,White,Not-in-family,Private,"(31.6, 38.9]"


In [6]:
adult_dataset = NamedDataset('adult',adult)

In [7]:

features_num = ['hours-per-week', 'capital-gain', 'capital-loss'] 
features_cat =  ['workclass', 'relationship', 'occupation','education'] 
protected_attributes = ['sex','race','age_bins', 'marital-status']
target = 'income-per-year'

p_att_description = [('sex',[1]),('race',['White']),('age_bins',['(38.9, 46.2]','(46.2, 53.5]','(53.5, 60.8]']),\
                     ('marital-status',['Married-civ-spouse'])]
model = [fsm.fair_classifier_adversarial]
technique = [fsm.xy_greater_xa]
etas = [1,10,20]
feat_dictionary= {'hours-per-week':5,'capital-gain':10,'capital-loss':5,'workclass':7,\
                  'education':16, 'relationship':6, 'occupation':14 }
#greater= False #XA<XY
#k=5
 

thesis_param_grid = {'data':[adult_dataset],'features_cat': [features_cat], 'features_num':[features_num],\
                     'protected_attributes':[protected_attributes], 'p_att_description' : p_att_description,\
                     'target': [target], 'etas': [etas], 'feat_dictionary' : [feat_dictionary],'greater': [False,True], 'k':[5],\
                     'technique': technique, 'model': model}

my_expt = Experiment(fsm.main_exp_bf_func,thesis_param_grid)
batchname, successes ,fails = my_expt.run_batch(expt_name = 'case_study_adversarial1', name_func = name_function)


2023-02-06 05:30:02.984459: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-06 05:30:02.984486: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uri-cpu011): /proc/driver/nvidia/version does not exist
2023-02-06 05:30:02.984790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


{'age_bins': array(['occupation', 'education', 'capital-gain', 'capital-loss'],
      dtype=object), 'marital-status': array(['occupation', 'education', 'capital-gain'], dtype=object), 'race': array(['workclass', 'relationship', 'occupation', 'education',
       'hours-per-week', 'capital-gain', 'capital-loss'], dtype=object), 'sex': array(['workclass', 'education', 'hours-per-week', 'capital-gain',
       'capital-loss'], dtype=object)}


2023-02-06 05:30:03.404582: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


epoch 0; iter: 0; batch classifier loss: 0.722252; batch adversarial loss: 0.626882
epoch 0; iter: 200; batch classifier loss: 0.583717; batch adversarial loss: 0.703234
epoch 1; iter: 0; batch classifier loss: 0.531258; batch adversarial loss: 0.776778
epoch 1; iter: 200; batch classifier loss: 0.620358; batch adversarial loss: 0.660778
epoch 2; iter: 0; batch classifier loss: 0.680145; batch adversarial loss: 0.603920
epoch 2; iter: 200; batch classifier loss: 0.471300; batch adversarial loss: 0.654809
epoch 3; iter: 0; batch classifier loss: 0.464050; batch adversarial loss: 0.636117
epoch 3; iter: 200; batch classifier loss: 0.406572; batch adversarial loss: 0.609082
epoch 4; iter: 0; batch classifier loss: 0.462728; batch adversarial loss: 0.676123
epoch 4; iter: 200; batch classifier loss: 0.368723; batch adversarial loss: 0.648064
epoch 5; iter: 0; batch classifier loss: 0.492015; batch adversarial loss: 0.605022
epoch 5; iter: 200; batch classifier loss: 0.506650; batch adversa

In [8]:
my_results = ExperimentResult('results/'+ 'case_study_adversarial12023_02_06_05_30_02_369431/')
my_results.get_result_names()

['adult_xy_greater_xa_fair_classifier_adversarial_age_bins_False_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_age_bins_True_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_marital-status_False_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_marital-status_True_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_race_False_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_race_True_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_sex_False_5',
 'adult_xy_greater_xa_fair_classifier_adversarial_sex_True_5']

In [9]:
casestudy_adversarial1 = my_results.stack_results()

In [10]:
casestudy_adversarial1.to_csv('casestudy_adversarial1.csv')